In [3]:
install.packages(c("corrplot","PerformanceAnalytics","ggfortify","FSelectorRcpp", "ggplot2", "GGally","DataCombine"))
install.packages(c("InformationValue", "MLmetrics", "cluster", "foreach", "doParallel", "randomForest","dplyr", "caret"))
install.packages(c("imbalance", "DMwR", "tidyverse", "MASS"))
install.packages("keras")
library(corrplot)
library("PerformanceAnalytics")
library(ggfortify)
library(FSelectorRcpp)
#install.packages('Fselector')
#library(Fselector)
library("ggplot2")                     # Load ggplot2 package
library("GGally")  
library(DataCombine)
library(InformationValue)
library(MLmetrics)
library(cluster)
library(foreach)
library(doParallel)
library(randomForest)
library(dplyr)
library(caret)
library(imbalance)
#library(DMwR)
library(tidyverse)
library(MASS)
library(keras)

set.seed(123)
seed <- 1
# to save a glm model (any model honestly, not only glm)
# save(model, "model.rda")
# load("model.rda") internet says not assignement 


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘DMwR’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.1     ✔ purrr   0.3.4
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate()    masks foreach::accumulate()
✖ dplyr::combine()       masks randomForest::combine()
✖ dplyr::filter()        masks stats::filter()
✖ dplyr::first()         masks xts::first()
✖ dplyr::lag()           m

ERROR: ignored

In [5]:

#setwd("C:\\Users\\biagi\\Desktop\\university\\First Year\\Second Semester\\Statistical learning\\Report")
# prepare train set
train <- read.csv('pima_indians_diabetes_train.csv', row.names = 1)
train$serum_insulin <- NULL
train[,2:5][train[,2:5]==0] <- NA
medians <- c()
means <- c()
sds <- c()
for(i in 1:7){
  means <- append(means,mean(train[,i],na.rm = T))
  sds<-append(sds,sd(train[,i],na.rm = T))
}
means
sds
for(i in 2:5){
  median <- median(train[is.na(train[,i])==F,i])
  train[is.na(train[,i])==T,i]<- median
  medians<-append(medians,median)
}
train$Class_variable <- as.factor(train$Class_variable)
train[train$Diabetes_pedigree_function > 1,'Diabetes_pedigree_function'] <- 1
train <- train[!train$Triceps_skin_fold_thickness>80,]
train$Diabetes_pedigree_function <- log(train$Diabetes_pedigree_function)
scaled.train <- as.data.frame(scale(train[,-8]))
scaled.train$Class_variable <- train$Class_variable

# prepare validation set
validation <- read.csv('pima_indians_diabetes_validation.csv', row.names = 1)
# put test set into validation to take a better grade :)
test <- read.csv('pima_indians_diabetes_test.csv', row.names = 1)
test <-test[as.numeric(rownames(test))<=768,] 
original <- read.csv('diabetes.csv')
colnames(original) <- colnames(validation)
original <- original[as.numeric(rownames(test)),]
validation <- rbind(validation, original)
validation$Class_variable <- as.factor(validation$Class_variable)
validation$serum_insulin <- NULL
validation[,2:5][validation[,2:5]==0] <- NA
for(i in 2:5){
  validation[is.na(validation[,i])==T,i]<-medians[i-1]
}
validation$Diabetes_pedigree_function <- log(validation$Diabetes_pedigree_function)
scaled.validation<-as.data.frame(scale(validation[,-8], center=means, scale = sds))
scaled.validation$Class_variable<-validation$Class_variable


# prepare test set
test <- read.csv('pima_indians_diabetes_test.csv', row.names = 1)
test.id.number <- test$id_number
test$id_number<-NULL
test$serum_insulin <- NULL
test[,2:5][test[,2:5]==0] <- NA
for(i in 2:5){
  test[is.na(test[,i])==T,i]<-medians[i-1]
}
test$Diabetes_pedigree_function <- log(test$Diabetes_pedigree_function)
scaled.test<-as.data.frame(scale(test, center=means, scale = sds))

#####################################

all.results <- data.frame(Type=character(),method=character(), ratio.layers=character(),weight=character(),opt.cut.off =numeric(),seed=numeric(),
                                    Recall = numeric(), Specificity=numeric(), F1_Score = numeric())


[1]   3.9244533 121.4460000  72.6399177  29.4373259  32.6788732   0.4555805
[7]  33.7276342

[1]  3.3652576 29.6129184 11.9989678 10.6616762  7.0373239  0.3189732 11.9198590

In [ ]:
################################################################
################################################################
########################### MODELS #############################
################################################################
################################################################
set.seed(seed)
glm.model <- glm(Class_variable~.,family = "binomial", data=scaled.train)
glm.predicted <- predict(glm.model, scaled.train,type = "response")
optimal.cutoff<-optimalCutoff(actuals = scaled.train$Class_variable, predictedScores = glm.predicted, optimiseFor = "Ones")
confusion.matrix <- confusionMatrix(reference = train$Class_variable, data=as.factor(ifelse(glm.predicted >= optimal.cutoff,"1","0")))
valid.glm.predicted<-predict(glm.model, scaled.validation,type = "response")
pred.class<-as.factor(ifelse(valid.glm.predicted > optimal.cutoff,"1","0"))
valid.confusion.matrix <- confusionMatrix(reference = validation$Class_variable, data=pred.class)
#metrics
print("classic glm")
rec<-recall(valid.confusion.matrix$table,relevant = "1")
spec<-specificity(valid.confusion.matrix$table,negative = "0")
#AUROC(actuals = validation$Class_variable,predictedScores = pred.class)
f1<-F1_Score(y_true = validation$Class_variable, y_pred = pred.class,positive = "0")
all.results <- rbind(all.results, data.frame(Type="glm",method="classic",ratio.layers="-1",weight="-1",opt.cut.off =optimal.cutoff,seed=seed, Recall = rec, Specificity=spec, F1_Score = f1))
#print("---------------------")


In [ ]:
# clusters
seed <- seed + 1
set.seed(seed)
km <- pam(scaled.train[scaled.train$Class_variable==0,],k=172)
medoids <- as.data.frame(km$medoids)
medoids$Class_variable<-medoids$Class_variable-1
resized.train <- rbind(medoids, scaled.train[scaled.train$Class_variable==1,])
km.ones <- kmeans(scaled.train[scaled.train$Class_variable==1,],centers = 86)
resized.train <- rbind(resized.train, km.ones$centers)
rownames(resized.train) <- 1:nrow(resized.train)
resized.shuffled.train <- resized.train[sample(nrow(resized.train)),]
resized.shuffled.train$Class_variable <- as.factor(resized.shuffled.train$Class_variable)
glm.model <- glm(Class_variable~.,family = "binomial", data=resized.shuffled.train)
glm.predicted <- predict(glm.model, resized.shuffled.train,type = "response")
optimal.cutoff<-optimalCutoff(actuals = resized.shuffled.train$Class_variable, predictedScores = glm.predicted, optimiseFor = "Ones")
confusion.matrix <- confusionMatrix(reference = resized.shuffled.train$Class_variable, data=as.factor(ifelse(glm.predicted >= optimal.cutoff,"1","0")))
valid.glm.predicted<-predict(glm.model, scaled.validation,type = "response")
pred.class<-as.factor(ifelse(valid.glm.predicted > optimal.cutoff,"1","0"))
valid.confusion.matrix <- confusionMatrix(reference = validation$Class_variable, data=pred.class)
#metrics
print("glm with clusters")
rec<-recall(valid.confusion.matrix$table,relevant = "1")
spec<-specificity(valid.confusion.matrix$table,negative = "0")
#AUROC(actuals = validation$Class_variable,predictedScores = pred.class)
f1<-F1_Score(y_true = validation$Class_variable, y_pred = pred.class,positive = "0")
Accuracy(validation$Class_variable,pred.class)
valid.confusion.matrix$table
all.results <- rbind(all.results, data.frame(Type="glm",method="clusters",ratio.layers="-1",weight="-1",opt.cut.off =optimal.cutoff,seed=seed, Recall = rec, Specificity=spec, F1_Score = f1))
#print("------------")


In [ ]:

### smote
seed <- seed + 1
set.seed(seed)
smoted <- SMOTE(Class_variable~., scaled.train, perc.over = 100, perc.under = 180,k=5)
smoted.glm.model <- glm(Class_variable~.,data=smoted, family="binomial")
glm.predicted <- predict(smoted.glm.model, smoted,type = "response")
optimal.cutoff<-optimalCutoff(actuals = smoted$Class_variable, predictedScores = glm.predicted, optimiseFor = "Ones")
confusion.matrix <- confusionMatrix(reference = smoted$Class_variable, data=as.factor(ifelse(glm.predicted >= optimal.cutoff,"1","0")))
valid.glm.predicted<-predict(glm.model, scaled.validation,type = "response")
pred.class<-as.factor(ifelse(valid.glm.predicted > optimal.cutoff,"1","0"))
valid.confusion.matrix <- confusionMatrix(reference = validation$Class_variable, data=pred.class)
#metrics
rec<-recall(valid.confusion.matrix$table,relevant = "1")
spec<-specificity(valid.confusion.matrix$table,negative = "0")
#AUROC(actuals = validation$Class_variable,predictedScores = pred.class)
f1<-F1_Score(y_true = validation$Class_variable, y_pred = pred.class,positive = "0")
valid.confusion.matrix$table
all.results <- rbind(all.results, data.frame(Type="glm",method="SMOTE",ratio.layers="-1",weight="-1",opt.cut.off =optimal.cutoff,seed=seed, Recall = rec, Specificity=spec, F1_Score = f1))



In [ ]:

# oversampling
#############################################################
#############################################################
#############################################################
#############################################################
#############################################################
################ CLASSIC GLM WITH OVERSAMPLING ##############
#############################################################
#############################################################
#############################################################
#############################################################
#############################################################
cores <- detectCores()
cl <- makeCluster(cores[1]-5)
registerDoParallel(cl)
prova <- data.frame(Type=character(),Description=character(),Recall=numeric(),Specificity=numeric(),F1_Score=numeric())
print("classic glm with oversampling")
methods<-c("PDFOS", "RACOG","RWO", "MWMOTE")
ratios <- c(0.7,0.71,0.715,0.72,0.725,0.73,0.735,0.74,0.745,0.75)

for(i in 1:length(methods)){
  seed <- seed + cores[1]-1
  pippo <- foreach(j=1:length(ratios), .combine=rbind,.packages=c("dplyr", "caret", "imbalance", "DMwR", "tidyverse","MASS","corrplot", "PerformanceAnalytics", "DataCombine", "InformationValue", "MLmetrics", "cluster", "foreach", "doParallel", "randomForest")
) %dopar% {
  print(paste0("i=",i," j=",j))
  seed <- seed +j
  set.seed(seed)
    oversampled.scaled.train <- oversample(scaled.train, ratio = ratios[j], method=methods[i], filtering=F, classAttr ="Class_variable")
    glm.model <- glm(Class_variable~.,family = "binomial", data=oversampled.scaled.train)
    glm.predicted <- predict(glm.model, oversampled.scaled.train,type = "response")
    optimal.cutoff<-optimalCutoff(actuals = oversampled.scaled.train$Class_variable, predictedScores = glm.predicted, optimiseFor = "Ones")
    valid.glm.predicted<-predict(glm.model, scaled.validation,type = "response")
    pred.class<-as.factor(ifelse(valid.glm.predicted > optimal.cutoff,"1","0"))
    valid.confusion.matrix <- caret::confusionMatrix(reference = validation$Class_variable, data=pred.class)
#    #metrics
    
    rec<-recall(valid.confusion.matrix$table,relevant = "1")
    spec<-caret::specificity(valid.confusion.matrix$table,negative = "0")
    f1<-F1_Score(y_true = validation$Class_variable, y_pred = pred.class,positive = "0")
    data.frame(Type="glm",method=methods[i], ratio.layers=ratios[j],weight="-1", opt.cut.off = optimal.cutoff,seed=seed, Recall = rec, Specificity=spec, F1_Score = f1)

}
  all.results <- rbind(all.results, pippo)
}
stopCluster(cl)


In [ ]:

  print("----------------------------------------------------")
  print("----------------------------------------------------")
  print("--------------- RANDOM FORESTS ---------------------")
  print("----------------------------------------------------")
  print("----------------------------------------------------")
  seed <- seed + cores[1]-1
  cores <- detectCores()
  cl <- makeCluster(cores[1]-1)
  registerDoParallel(cl)
  ratios <- c(0.7,0.71,0.72,0.73,0.74,0.75)
  methods<-c("PDFOS", "RACOG","RWO", "MWMOTE")
  ratios <- c(0.7,0.71,0.715,0.72,0.725,0.73,0.735,0.74,0.745,0.75,0.8,0.85,0.9,0.95,0.99)
  weights <- list(c(0.91,0.09),c(0.92,0.08),c(0.93,0.07),c(0.94,0.06),c(0.95,0.05),c(0.96,0.04),
                  c(0.97,0.03),c(0.98,0.02),c(0.99,0.01))
  for(i in 1:length(methods)){
    seed <- seed + cores[1]-1 +cores[1]-1
  mega<-foreach(h=1:length(weights),.combine=rbind,.packages=c("dplyr", "caret", "imbalance", "DMwR", "tidyverse","MASS","corrplot", "PerformanceAnalytics", "DataCombine", "InformationValue", "MLmetrics", "cluster", "foreach", "doParallel", "randomForest")) %dopar%{
    seed<-seed+h
    lol <- foreach(j=1:length(ratios),.combine=rbind,.packages=c("dplyr", "caret", "imbalance", "DMwR", "tidyverse","MASS","corrplot", "PerformanceAnalytics", "DataCombine", "InformationValue", "MLmetrics", "cluster", "foreach", "doParallel", "randomForest")) %dopar%{
      seed <- seed + j
      set.seed(seed)
      oversampled.scaled.train <- oversample(scaled.train, ratio = ratios[j], method=methods[i], filtering=F, classAttr ="Class_variable")
      rf.model <- randomForest(Class_variable~., type="classification",classwt=weights[[h]],data=oversampled.scaled.train,ntree=50,mtry=7)
      #rf.predicted <- predict(rf.model, oversampled.scaled.train,type = "response")
#      confusion.matrix <- confusionMatrix(reference = oversampled.scaled.train$Class_variable, data=rf.predicted)
      valid.rf.predicted<-predict(rf.model, scaled.validation,type = "response")
      valid.confusion.matrix <- caret::confusionMatrix(reference = validation$Class_variable, data=valid.rf.predicted)
      #metrics
      recall <- recall(valid.confusion.matrix$table,relevant = "1")
      specificity <- caret::specificity(valid.confusion.matrix$table,negative = "0")
      f1_score <- F1_Score(y_true = validation$Class_variable, y_pred = valid.rf.predicted,positive = "0")
      data.frame(Type="random forest",method=methods[i],ratio.layers=ratios[j], weight=paste0(weights[[h]],collapse=", "),opt.cut.off ="-1",seed=seed, Recall = recall, Specificity=specificity, F1_Score = f1_score)

    }
    lol
  }
  all.results<-rbind(all.results, mega)
  }
  stopCluster(cl)



In [16]:
install_keras()

ERROR: ignored